# Stock Prediction 
In diesem Notebook soll mit verschiedenen Deep Learning Methoden der Kurs einer Aktie vorhergesagt werden. <br>
<ins>Ziele</ins> : <br>
- Implementierung von LSTM Algorithmus zur Vorhersage von einem Tag mit den vorherigen 100 Tagen als Input
- Implementierung von LSTM Algorithmus zur Vorhersage von 4 Tagen mit den vorherigen 100 Tagen als Input
- Vorhersage von einem Tag mithilfe von Linearer Regression
- Vorhersage von 1 Tag mithilfe von 1D Convolution
- Einfache Vorhersage mit einer Baseline indem der naechste Tag den Wert des vorherigen tages annimmt
- Vergleich der Performance all dieser Algorithmen in Bezug auf Accuracy und Anwendbarkeit
- bestehen des DL Moduls


## Laden der notwendigen Libraries


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import os
import datetime



# load required libraries:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.layers import LSTM, Dense, Convolution2D, MaxPooling2D, Flatten , Activation, Lambda, Convolution1D
from tensorflow.keras.utils import to_categorical 

from sklearn.preprocessing import MinMaxScaler

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

import matplotlib.pyplot as plt
#%matplotlib qt
plt.style.use('default')

path = './kaggle/Input/Stocks/'
path='archive/Data/Stocks/'
#path='Stocks/' #geschäftlich
stocks = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]



Individuelles auswaehlen verschiedner Aktien, nach Bedarf des Bedieners.<br>
Als Hinweis: <br>
- Tesla - tsla
- Microsoft - msft
- Apple - AAPl

In [ ]:
while True:
    stock = input('Pick a stock:')
    stock += '.us.txt'
    if stock in stocks:
        print('Found stock')
        data = pd.read_csv(path + stock)
        df = pd.read_csv(path + stock)
        break
    else:
        print('Stock not found')
        continue


In [ ]:
data.head()
#np.shape(data)


In [ ]:
data.describe()
type(data)
dataTypeSeries = data.dtypes
print('Data type of each column of Dataframe :')
print(dataTypeSeries)


In [ ]:
data.drop('OpenInt', axis=1, inplace=True)
df.drop('OpenInt', axis=1, inplace=True)
#df = data.tail(260)
#df = data
print(data, df)


## Anzeige der Aktie 

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=data.Date, 
                                     open=data.Open, 
                                     high=data.High, 
                                     low=data.Low, 
                                     close=data.Close)])
fig.show()


## Funktionen
Nachfolgende ist die Implementierung verschiedener Funktionen. 

Liste aller files im Ordner

In [ ]:

def files_list(data_path):
    files=[]
    for file in os.listdir(data_path):
        if file.endswith('.txt'):
            
            files.append(os.path.join(data_path, file))
    return files


Einlesen der Dateien.

In [ ]:
def txt_to_df(path):
    df=pd.read_csv(path,sep=',')
    df['Date']=pd.to_datetime(df.Date,format='%Y-%m-%d')
    df=df[['Date','Close']]
    scaler=MinMaxScaler(feature_range=(0,1))
    df['Close']=scaler.fit_transform(np.array(df.Close).reshape(-1,1))
    return df,scaler


Spalten und Transformieren der Datensets in fuer den Algorithmus angepasste Form.

In [ ]:
def create_dataset(dataset, time_step=1):
    print(time_step)
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset.Close[i:(i+time_step)]
        dataX.append(a)
        dataY.append(dataset.Close[i + time_step])
    return np.array(dataX), np.array(dataY)

def create_dataset_10(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-4):
        a = dataset.Close[i:i+time_step]
        dataX.append(a)
        
        b=dataset.Close[i+time_step:i+time_step+4]
        dataY.append(b)
        
    return np.array(dataX), np.array(dataY)


Nachfolgend werden die Funktionen aufgerufen und die jeweiligen Daten eingelesen.<br>
<ins>Wichtig:</ins> der Dateipfad muss an das jeweilige Verzeichnis angepasst werden. <br>
<br>
Als Spaltung der test und Trainingsdaten wurde das Jahr 2016 verwendet. 

In [ ]:

#data_path='archive/Data/Stocks/'
data_path='kaggle/Input/Stocks/'
data_path='archive/Data/Stocks/'
#data_path='Stocks/'
file_list=files_list(data_path)

ind=[i for i, s in enumerate(file_list) if 'tsla' in s]

data,scaler=txt_to_df(data_path+'tsla.us.txt')

# =============================================================================
# test=data.loc[data.Date=='2015-04-01'].index[0]
# data=data[test:]
# data.reset_index(inplace=True)
# =============================================================================

cut_off=data.loc[data.Date=='2016-03-01'].index[0]

train_data=data[:cut_off]
test_data=data[cut_off:]
test_data.reset_index(inplace=True)



Um 1 Tag vorherzusagen, werden die letzten 100 Tage vo diesem Tag als Input fuer den Algorithmus genommen. <br>
Man koennte auch weniger Time Steps als Input nehmen. Auch bei der 4 Tages vorhersage wurden die letzten 100 <br>
Tage als Input ausgesucht, um eine gewisse Vergleichbarkeit mit der Ein-Tages-Vorhersage gewaehrleisten zu koennen.

In [ ]:

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
#X_train


In [ ]:

time_step = 100
X_train_10, y_train_10 = create_dataset_10(train_data, time_step)
X_test_10, y_test_10 = create_dataset_10(test_data, time_step)

X_train_10 = X_train_10.reshape(X_train_10.shape[0],X_train_10.shape[1] , 1)
X_test_10 = X_test_10.reshape(X_test_10.shape[0],X_test_10.shape[1] , 1)
#X_train



## Implementierung der 4 Tage Vorhersage mit LSTM- Algorithmus


In [ ]:
model_lstm_10=Sequential()
model_lstm_10.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
#model.add(Dropout=0.2)
model_lstm_10.add(LSTM(50,return_sequences=True))
model_lstm_10.add(LSTM(50))
model_lstm_10.add(Dense(4))
model_lstm_10.compile(loss='mean_squared_error',optimizer='adam')


#model_lstm_10.fit(X_train_10,y_train_10,epochs=100,batch_size=32)

if os.path.exists('./StockModel_LSTM_10_02.h5'):
    print("Loading model")
    model_lstm_10 = tf.keras.models.load_model('./StockModel_LSTM_10_02.h5')
else:
    history=model_lstm_10.fit(X_train_10, y_train_10, 
                      epochs=100, 
                      batch_size=32, 
                      verbose=1)
    model_lstm_10.save('./StockModel_LSTM_10_02.h5')


In [ ]:


train_pred=[]
for i in range(0,len(X_train_10),1):
    data=X_train_10[i]
    data=data.reshape((1, len(data), 1))
    lstm_train_predict=model_lstm_10.predict(data)
    train_pred.append(lstm_train_predict)

test_pred=[]
for i in range(0,len(X_test_10),1):
    data=X_test_10[i]
    data=data.reshape((1, len(data), 1))
    lstm_test_predict=model_lstm_10.predict(data)
    test_pred.append(lstm_test_predict)


In [ ]:
test_pred=np.array(test_pred)
train_pred=np.array(train_pred)


### Vergleich der 4-Tage Vorhersage zum eigentlichen Aktienkurs

In [ ]:
fig,ax=plt.subplots()
ax.plot(test_pred.flatten(),color='red',label='Test data pred')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
ax.plot(y_test_10.flatten(),color='blue', label= 'Test data')
plt.legend(loc="upper left")

fig1,ax=plt.subplots()
ax.plot(train_pred.flatten(),color='red',label='Train data prediction')
ax.plot(y_train_10.flatten(),color='blue', label= 'Training data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")



## Implementierung 1-Tag Vorhersage mit LSTM- Algorithmus

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(100,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')


#model.fit(X_train,y_train,epochs=10,batch_size=32)


if os.path.exists('./StockModel_LSTM_02.h5'):
    print("Loading model")
    model = tf.keras.models.load_model('./StockModel_LSTM_02.h5')
else:
    history=model.fit(X_train, y_train, 
                      epochs=100, 
                      batch_size=32, 
                      verbose=1)
    model.save('./StockModel_LSTM_02.h5')



### Vergleich der 1 -Tag Vorhersage zum eigentlichen Aktienkus

In [ ]:
lstm_train_predict=model.predict(X_train)
lstm_test_predict=model.predict(X_test)

fig,ax=plt.subplots()
ax.plot(lstm_test_predict,color='red',label='Test data pred')
ax.plot(y_test,color='blue', label= 'Test data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")

fig1,ax=plt.subplots()
ax.plot(lstm_train_predict,color='red',label='Train data prediction')
ax.plot(y_train,color='blue', label= 'Training data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")
# =============================================================================
# y_train=scaler.inverse_transform(np.array(y_train).reshape(-1,1))
# =============================================================================

# =============================================================================
# train_predict=scaler.inverse_transform(train_predict)
# test_predict=scaler.inverse_transform(test_predict)
# =============================================================================



#### Accuracy
Laueft nur problemlos durch, wenn der Algorithmus neu gebildet wird und nicht geladen.

In [ ]:
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['accuracy'], color = 'red')
#plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend(['Train','Accuracy'])





#### 1.te Versuch der 1D Convolution

In [ ]:
# =============================================================================
# import tensorflow as tf
# from keras.layers import Flatten
# from keras.layers.convolutional import Conv1D
# from keras.layers.convolutional import MaxPooling1D
# 
# model_1Dconv = Sequential()
# ks = 3
# model_1Dconv.add(Convolution1D(filters=32, activation="relu", kernel_size=ks, padding='causal', input_shape=(100, 1)))
# #model_1Dconv.add(Convolution1D(filters=32, activation="relu", kernel_size=ks, padding='causal'))
# #model_1Dconv.add(Convolution1D(filters=32, activation="relu", kernel_size=ks, padding='causal'))
# model_1Dconv.add(Convolution1D(filters=32, activation="relu", kernel_size=ks, padding='causal'))
# model_1Dconv.add(MaxPooling1D(pool_size=2))
# model_1Dconv.add(Flatten())
# #model_1Dconv.add(Dense(32))
# model_1Dconv.add(Dense(1))
# 
# 
# model_1Dconv.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
# model_1Dconv.summary()
# =============================================================================

# =============================================================================
# history = model_1Dconv.fit(X_train, y_train,
#                     epochs=100,
#                     batch_size=32)
# =============================================================================


# =============================================================================
# print(history.history.keys())
# plt.plot(history.history['loss'])
# plt.plot(history.history['accuracy'], color = 'red')
# #plt.plot(history.history['val_loss'])
# 
# plt.xlabel('Epoch')
# plt.ylabel('Mean Absolute Error Loss')
# plt.title('Loss Over Time')
# plt.legend(['Train','Accuracy'])
# =============================================================================


# Prediction 1D CNN
# =============================================================================
# train_predict=model_1Dconv.predict(X_train)
# test_predict=model_1Dconv.predict(X_test)
# 
# fig,ax=plt.subplots()
# ax.plot(test_predict,color='red',label='Test data pred')
# ax.plot(y_test,color='blue', label= 'Test data')
# ax.set_ylabel('norm. USD $')
# ax.set_xlabel('samples')
# plt.legend(loc="upper left")
# 
# fig1,ax=plt.subplots()
# ax.plot(train_predict,color='red',label='Train data prediction')
# ax.plot(y_train,color='blue', label= 'Training data')
# ax.set_ylabel('norm. USD $')
# ax.set_xlabel('samples')
# plt.legend(loc="upper left")
# =============================================================================





## Implementierung der 1-Tag Vorhersage mit 1D- Convolution

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

#1 D Conv Prediction 
conv_1d_model = Sequential()
conv_1d_model.add(Conv1D(filters=64, kernel_size=10, activation='relu', input_shape=(100, 1)))
conv_1d_model.add(MaxPooling1D(pool_size=2))
conv_1d_model.add(Flatten())
conv_1d_model.add(Dense(50, activation='relu'))
conv_1d_model.add(Dense(1))
conv_1d_model.compile(optimizer='adam', loss='mse')

if os.path.exists('./StockModel_1CNN_02.h5'):
    print("Loading model")
    conv_1d_model= tf.keras.models.load_model('./StockModel_1CNN_02.h5')
else:
    history=conv_1d_model.fit(X_train, y_train, 
                      epochs=50, 
                      batch_size=32, 
                      verbose=1)
    conv_1d_model.save('./StockModel_1CNN_02.h5')


train_predict=conv_1d_model.predict(X_train)
test_predict=conv_1d_model.predict(X_test)

fig,ax=plt.subplots()
ax.plot(test_predict,color='red',label='Test data pred')
ax.plot(y_test,color='blue', label= 'Test data')
plt.legend(loc="upper left")

fig1,ax=plt.subplots()
ax.plot(train_predict,color='red',label='Train data prediction')
ax.plot(y_train,color='blue', label= 'Training data')
plt.legend(loc="upper left")




## Impelementierung der 1-Tag Vorhersage durch Lineare Regression

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers


In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)
#X_train


In [ ]:
#built the neural network architecture
model_lin= Sequential()
model_lin.add(Dense(15, input_dim=100, activation='relu'))
#model_lin.add(Dense(7, activation='relu'))
#model_lin.add(Dense(3, activation='relu'))
model_lin.add(Dense(1, activation='linear'))
model_lin.summary()


In [ ]:
model_lin.compile(optimizer='adam', loss='mse')


In [ ]:
#train the neural network on the train dataset
history= model_lin.fit(X_train, y_train, epochs=50, validation_split=0.2)


In [ ]:
#plot the loss and validation loss of the dataset
history_lin = pd.DataFrame(history.history)
plt.plot(history_lin['loss'], label='loss')
plt.plot(history_lin['val_loss'], label='val_loss')

plt.legend()


In [ ]:
#evaluate the model
model_lin.evaluate(X_test, y_test, batch_size=128)


Vergleich der 1D- Convolution Vorhersage mit dem eigentlichen Aktienkurs

In [ ]:
y_pred = model_lin.predict(X_test)

a = plt.axes(aspect='equal')
plt.scatter(y_test, y_pred, color = 'red')
plt.xlabel('Input')
plt.ylabel('Predicted values')
plt.title('True and predicted values')
plt.plot([0.5, 1], [0.5, 1])

train_predict= model_lin.predict(X_train)
test_predict=model_lin.predict(X_test)

fig,ax=plt.subplots()
ax.plot(test_predict,color='red',label='Test data pred')
ax.plot(y_test,color='blue', label= 'Test data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")

fig1,ax=plt.subplots()
ax.plot(train_predict,color='red',label='Train data prediction')
ax.plot(y_train,color='blue', label= 'Training data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")



## Implementierung der Baseline
Als Vorhersage wird immer der Wert des Vortages genommen.

In [ ]:
#als prediction einfach immer den Wert vom Vortag nehmen
from scipy.ndimage.interpolation import shift
base_train_predict=shift(y_train,1,cval=y_train[0])
base_test_predict=shift(y_test,1,cval=y_train[0])

base_test_predict[0]=base_train_predict[-1]


fig,ax=plt.subplots()
ax.plot(base_test_predict,color='red',label='Test data pred')
ax.plot(y_test,color='blue', label= 'Test data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")

fig1,ax=plt.subplots()
ax.plot(base_train_predict,color='red',label='Train data prediction')
ax.plot(y_train,color='blue', label= 'Training data')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")


In [ ]:
y_fehl_b=abs((y_test-base_test_predict))
y_fehl_lstm=abs((y_test-lstm_test_predict.flatten()))

fig1,ax=plt.subplots()
ax.plot(y_fehl_b,color='red',label='Differenz base zu original')
ax.plot(y_fehl_lstm,color='blue', label= 'Differenz lstm zu original')
ax.set_ylabel('norm. USD $')
ax.set_xlabel('samples')
plt.legend(loc="upper left")


## Kalkulation der Performance der verschiedenen Modelle und deren Vergleich

In [ ]:
# Model Performance Calculation
model_performance = {}
# Performance measure Linear model Baseline
model_performance['Lin Reg'] = model_lin.evaluate(X_test, y_test, verbose=0)
# Performance measure 1D CNN
model_performance['1D CNN'] = conv_1d_model.evaluate(X_test, y_test, verbose=0)
# Performance measure LSTM
model_performance['LSTM'] = model.evaluate(X_test, y_test, verbose=0)
#
model_performance['LSTM_4'] = model.evaluate(X_test_10, y_test_10, verbose=0)




In [ ]:
# Creating model performance plot
models = list(model_performance.keys())
values = list(model_performance.values())

In [ ]:
fig = plt.figure(figsize = (5, 3))
 
# creating the bar plot
plt.bar(models, values, color ='blue',
        width = 0.1)
 
plt.xlabel("Modelle")
plt.ylabel("Mean Square Error")
plt.title("Modellvergleich")
plt.show()


## Verbesserungsmoeglichkeiten
- Lstm mit Multi Step Prediction durch Encoder-Decoder System 
- Trainieren der Daten mit mehr als nue einem Feature. Hier wurde nur der Closing Wert der Aktie am Tag verwendet. Es haetten auch weitere Features, wie Open-Wert verwendet werden koennen.
- Weniger oder mehr Tage als Input fuer die Vorhersage verwenden
- Test und Trainingsdaten Aufspaltung 